In [3]:
from re import T
import pandas as pd
import os
import numpy as np
import math

# Matching dictionary for team names between events and tracking data
TEAM_NAMES = {'Canada': 'Olympic (Women) - Canada', 
              'USA': 'Olympic (Women) - United States',
              'Finland': 'Olympic (Women) - Finland',
              'ROC': 'Olympic (Women) - Olympic Athletes from Russia',
              'Switzerland': 'Olympic (Women) - Switzerland'}

# Max number of frames to skip in velocity calculations
MAX_FRAME_JUMP = 5

In [4]:
# This is for convenience to match our wd just in case the notebook is located elsewhere
os.chdir('/mnt/c/Users/thepi/OneDrive/Documents/Python/Hockey/Big-Data-Cup-2022-Private')
# os.chdir('C:/Users/Paula/Desktop/Big-Data-Cup-2022-Private')

In [6]:
def get_names_date(game):
    # get's team names and date from the game file name
    spl = game.split(' ')
    date_comps = spl[0].split('-')
    date = '{}/{}/{}'.format(*[dt.strip('0') for dt in date_comps[::-1]])
    team_1 = TEAM_NAMES[spl[1]]
    team_2 = TEAM_NAMES[spl[3]]
    
    return team_1, team_2, date
    
def get_speed(track):
    # calculate velocity using displacement between frames
    assert (track['track_id'] == track['track_id'].iloc[0]).all()
    try:
        frame_diff = np.diff(track['frame_id'])
        nonvalid_speed = np.concatenate([[True],frame_diff>MAX_FRAME_JUMP])
        next_valid_speed = np.concatenate([frame_diff<=MAX_FRAME_JUMP,[False]])
        repl = nonvalid_speed & next_valid_speed
        idx = np.where(repl)[0]+1
        
        x_spd = np.concatenate([[np.nan],np.diff(track['x_ft'])/frame_diff])*30
        y_spd = np.concatenate([[np.nan],np.diff(track['y_ft'])/frame_diff])*30
        
        x_spd[repl] = x_spd[idx]
        y_spd[repl] = y_spd[idx]

    except Exception as e:
        print(np.diff(track['frame_id']))
        raise

    return(x_spd, y_spd)

In [31]:
def get_pp_tracking(power_play_info_file = 'pp_info.csv',
    play_by_play_data_file = 'pxp_womens_oly_2022_v2.csv',
    tracking_dir = 'data',
    pbp_dir = 'data'):
    '''
    Function for reading tracking and event data and matching relevant tracks to events
    '''
    play_by_play_data = pd.read_csv(os.path.join(tracking_dir,play_by_play_data_file))
    power_play_info = pd.read_csv(os.path.join(pbp_dir,power_play_info_file))

    #power_play_info['game_name'].loc[power_play_info['game_name']=='2022-02-14 USA at Finland'] = '2022-02-14 Finland at USA'
    power_play_info['relevant_events'] = np.nan

    # Big data frame for all games and power plays
    relevant_eventses = []

    # Run through all the games and powerplays
    for ix, pp in power_play_info.iterrows():
        relevant_events = None
        game = pp['game_name']
        # print(game)
        #if game!='2022-02-14 Finland at USA':
        
        # Read relevant files
        game_files = os.listdir(os.path.join(tracking_dir,game))
        tracking_data_name = os.path.join(tracking_dir,game,f'{game} P{pp["start_period"]} PP{pp["penalty_number"]}.csv')
        tracking_info_name = os.path.join(tracking_dir,game,f'videoShotsInfo_{game} P{pp["start_period"]} PP{pp["penalty_number"]}.csv')
        roster_info_name = os.path.join(tracking_dir, game, f'{game} roster.csv')
        roster_info = pd.read_csv(roster_info_name, index_col = 0)
        
        # Get team names
        away, _ , home = game.split(' ')[-3:]
        roster_info.loc[roster_info['team']=='away','team'] = away
        roster_info.loc[roster_info['team']=='home','team'] = home

        # mark goalies for each team that is playing (used later)
        goalie = {f'{rf["team"]}_{rf["jn"]}':rf['position']=='Goalie' for _,rf in roster_info.iterrows()}

        # read entire tracking file and calculate speeds and skating directions
        if os.path.split(tracking_data_name)[1] in game_files:
            print(tracking_data_name)
            tracking_data = pd.read_csv(tracking_data_name)
            # add column identifiyng whether each track belongs to a goal keeper
            tracking_data['goalie'] = [goalie[f'{tr["team_name"]}_{tr["jersey_number"]}'] if f'{tr["team_name"]}_{tr["jersey_number"]}' in goalie.keys() else False for _,tr in tracking_data.iterrows()]


            tracking_data['vel_x'] = np.nan
            tracking_data['vel_y'] = np.nan
            tracking_data['angle'] = np.nan
            tr_ids = tracking_data.track_id.unique()
            # Calculate velocity and skating angle
            for tr_id in tr_ids:
                idxs = tracking_data.track_id==tr_id
                track = tracking_data.loc[idxs]
                if len(track) <= 5:
                    continue
                x_spd,y_spd = get_speed(track)
                tracking_data.loc[idxs,'vel_x'] = x_spd
                tracking_data.loc[idxs,'vel_y'] = y_spd
                tracking_data['angle'] = np.arctan2(tracking_data['vel_y'],tracking_data['vel_x'])
            tracking_info = pd.read_csv(tracking_info_name)
        else:
            # Can't file this tracking file
            print('Tracking file missing:\n{}'.format(tracking_data_name))
            relevant_eventses.append(None)
            continue
        team_1, team_2, date = get_names_date(game)
       
        # filter relevant plays for current period and power play
        relevant_events = play_by_play_data.loc[(play_by_play_data['game_date'] == date) & 
                                    ((play_by_play_data['team_name'] == team_1) |
                                        (play_by_play_data['team_name'] == team_2))]

        relevant_events = relevant_events.loc[((relevant_events['period']>= pp['start_period']) 
                                                & (relevant_events['clock_seconds'] <= pp['start_game_clock_seconds'])) &
                                              ((relevant_events['period']<= pp['end_period']) & 
                                              (relevant_events['clock_seconds'] >= pp['end_game_clock_seconds']))  ]

        # Placeholders for time to next shot and time to next goal
        relevant_events.index = np.arange(0, len(relevant_events.index), dtype = 'int')
        relevant_events['time_to_shot'] = np.ones(len(relevant_events)) * np.inf
        relevant_events['time_to_goal'] = np.ones(len(relevant_events)) * np.inf
        shots = relevant_events.loc[relevant_events.event == 'Shot']
        goals = shots.loc[shots.event_successful == 't']
        # Calculating the time to next shot, goal
        while len(shots):
            ix = (relevant_events['clock_seconds']>=shots.iloc[0].clock_seconds) & (relevant_events['time_to_shot'] > 100000)
            relevant_events.loc[ix,'time_to_shot'] = relevant_events.loc[ix,'clock_seconds'] - shots.iloc[0].clock_seconds
            shots = shots.iloc[1:,]
        while len(goals):
            ix = (relevant_events['clock_seconds']>=goals.iloc[0].clock_seconds) & (relevant_events['time_to_goal'] > 100000)
            relevant_events.loc[ix,'time_to_goal'] = relevant_events.loc[ix,'clock_seconds'] - goals.iloc[0].clock_seconds
            goals = shots.iloc[1:,]
        
        # Matching names and jersey numbers
        pl1_jn = roster_info.loc[relevant_events['player_name'],'jn']
        pl1_jn.index = relevant_events.index

        pl2_exists = list(relevant_events['player_name_2'].notna())
        pl2_jn = roster_info.loc[relevant_events['player_name_2'][pl2_exists],'jn']

        pl2_jn.index = relevant_events.index[pl2_exists]
        relevant_events['Player_1_num'] = pl1_jn
        relevant_events['Player_2_num'] = pl2_jn
        
        tracks = []
        
        # Giving each event the tracks between the beginning and end of the event (if we have the end, otherwise just the frame of the event)
        for rw, event in relevant_events.iterrows():
            fr_id_1 = event['frame_id_1']
            if pd.isna(fr_id_1):
                tracks.append(None)
                continue
            fr_id_2 = event['frame_id_2']
            if pd.notna(fr_id_2):
                tracks.append(tracking_data.loc[(tracking_data['frame_id']>=fr_id_1) &
                                                     (tracking_data['frame_id']<=fr_id_2)])
            else:
                tracks.append(tracking_data.loc[tracking_data['frame_id']==fr_id_1])

        # Assign track to the relevant event            
        relevant_events['tracks'] = tracks

        # Add the current event to the final dataframe for this pp
        relevant_eventses.append(relevant_events)
    # Add a column of relevant events to the power play infor dataframe
    power_play_info['relevant_events'] = relevant_eventses
    return power_play_info


In [32]:
# Read tracking and event data
relevant_events = get_pp_tracking()
relevant_events.head()

data/2022-02-08 Canada at USA/2022-02-08 Canada at USA P1 PP1.csv
   frame_id  period  track_id team_id team_name  jersey_number        x_ft  \
0       301       1         1   Light    Canada              6  164.071705   
1       301       1         2    Dark       USA              2  187.278571   
2       301       1         3    Dark       USA             35  187.258524   
3       301       1         4   Light    Canada             19  195.732672   
4       301       1         5    Dark       USA             28  167.116300   

        y_ft  video_shot  game_seconds  goalie  
0  42.072340           1           377   False  
1  37.647639           1           377   False  
2  43.182641           1           377    True  
3  63.352362           1           377   False  
4  34.757970           1           377   False  


AttributeError: 'int' object has no attribute 'head'

In [9]:
# Example of tracking data for one event
relevant_events.loc[0,'relevant_events'].tracks.iloc[6]

,frame_id,period,track_id,team_id,team_name,jersey_number,x_ft,y_ft,video_shot,game_seconds,vel_x,vel_y,angle
301,344,1,3,Dark,USA,35,185.378708,44.201831,1,375,-3.084396,1.223622,2.763923
302,344,1,2,Dark,USA,2,178.061495,47.371741,1,375,-6.897018,7.126913,2.339803
303,344,1,4,Light,Canada,19,190.091633,69.393926,1,375,-9.628467,7.603144,2.473193
304,344,1,1,Light,Canada,6,169.637361,54.290513,1,375,-0.641787,7.721428,1.653723
305,344,1,7,Dark,USA,5,185.094211,61.938154,1,375,-12.411864,3.411144,2.873385
...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,370,1,7,Dark,USA,5,176.362805,61.602780,1,374,-0.583101,-7.261438,-1.650925
510,370,1,9,Light,Canada,23,131.460731,30.983528,1,374,-7.497990,0.437055,3.083369
511,370,1,3,Dark,USA,35,181.717333,45.429456,1,374,0.000000,0.000000,0.000000
512,370,1,13,Light,Canada,29,136.055357,68.309436,1,374,22.730955,4.214396,0.183322


In [10]:
# Save to JSON to read from R
relevant_events.to_json(r'data/BDC_2022_all_data.json')

In [11]:
# Filter for direct passes only

relevant_events2 = relevant_events.loc[~relevant_events['relevant_events'].isna()]
the_truth = (relevant_events2.loc[0,'relevant_events']['event_type'] == 'Direct')&(~relevant_events2.loc[0,'relevant_events']['tracks'].isna())
r0_direct = relevant_events2.loc[0,'relevant_events'].loc[the_truth]

for r in relevant_events2.index[1:]:
    r_the_truth = (relevant_events2.loc[r,'relevant_events']['event_type'] == 'Direct')&(~relevant_events2.loc[r,'relevant_events']['tracks'].isna())
    r_new_direct = relevant_events2.loc[r,'relevant_events'].loc[r_the_truth]
    r0_direct=pd.concat([r0_direct, r_new_direct])
r0_direct = r0_direct.reset_index(drop=True)

In [12]:
r0_direct

,game_date,season_year,team_name,opp_team_name,venue,period,clock_seconds,situation_type,goals_for,goals_against,...,event_detail_1,event_detail_2,event_detail_3,frame_id_1,frame_id_2,time_to_shot,time_to_goal,Player_1_num,Player_2_num,tracks
0,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,375,5 on 4,0,0,...,NaN,NaN,NaN,344.0,370.0,23.0,23.0,19,29.0,frame_id period track_id team_id team_n...
1,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,373,5 on 4,0,0,...,NaN,NaN,NaN,420.0,443.0,21.0,21.0,29,6.0,frame_id period track_id team_id team_...
2,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,366,5 on 4,0,0,...,NaN,NaN,NaN,605.0,639.0,14.0,14.0,6,29.0,frame_id period track_id team_id team_...
3,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,365,5 on 4,0,0,...,NaN,NaN,NaN,650.0,670.0,13.0,13.0,29,23.0,frame_id period track_id team_id team_...
4,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,363,5 on 4,0,0,...,NaN,NaN,NaN,701.0,718.0,11.0,11.0,23,19.0,frame_id period track_id team_id team_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,16/2/2022,2021,Olympic (Women) - Finland,Olympic (Women) - Switzerland,home,3,74,5 on 4,3,0,...,NaN,NaN,NaN,3569.0,3602.0,16.0,16.0,16,40.0,frame_id period track_id team_id t...
402,16/2/2022,2021,Olympic (Women) - Finland,Olympic (Women) - Switzerland,home,3,68,5 on 4,3,0,...,NaN,NaN,NaN,3742.0,3763.0,10.0,10.0,40,6.0,frame_id period track_id team_id t...
403,16/2/2022,2021,Olympic (Women) - Finland,Olympic (Women) - Switzerland,home,3,67,5 on 4,3,0,...,NaN,NaN,NaN,3781.0,3812.0,9.0,9.0,6,33.0,frame_id period track_id team_id t...
404,16/2/2022,2021,Olympic (Women) - Finland,Olympic (Women) - Switzerland,home,3,65,5 on 4,3,0,...,NaN,NaN,NaN,3835.0,3852.0,7.0,7.0,33,16.0,frame_id period track_id team_id t...


In [13]:
# Save direct passes to R to read later
r0_direct.to_json(r'data/BDC_2022_all_direct_wFrames.json')

In [14]:
truth2 = r0_direct.loc[0,'tracks']['frame_id']==r0_direct.frame_id_1[0]
r_frame_direct = r0_direct.loc[0,'tracks'].loc[truth2]
r_direct_oneFrame = []

for r in r0_direct.index:
    r_truth2 = r0_direct.loc[r,'tracks']['frame_id']==r0_direct.frame_id_1[r]
    r2_frame_direct = r0_direct.loc[r,'tracks'].loc[r_truth2]
    r_direct_oneFrame.append(r2_frame_direct)

r0_direct['tracks']=r_direct_oneFrame

In [16]:
r0_direct = r0_direct.reset_index(drop=True)

In [17]:
r0_direct.to_json(r'data/BDC_2022_passOnly.json')